In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型监控XGBoost模型

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> 在Colab中运行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_monitoring%2Fget_started_with_model_monitoring_xgboost.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_xgboost.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      在GitHub上查看
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_xgboost.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI Model Monitoring监控XGBoost模型。

了解更多关于[Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring)。

### 目标

在本笔记本中，您将学习如何使用Vertex AI模型监控服务来检测XGBoost模型的输入预测请求中的特征偏差和漂移。

本教程使用以下谷歌云ML服务：

- Vertex AI模型监控
- Vertex AI预测
- Vertex AI模型资源
- Vertex AI端点资源

执行的步骤包括：

- 下载预训练的XGBoost模型。
- 将预训练模型上传到Vertex AI模型注册表。
- 将模型资源部署到Vertex AI端点资源。
- 配置端点资源以进行模型监控：
  - 仅执行漂移检测 -- 无法访问训练数据。
  - 预定义输入模式，将特征别名映射到模型的未命名数组输入。
- 生成用于漂移的合成预测请求。

了解更多关于[Vertex AI模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 模型

本教程使用的模型是一个预训练的 XGBoost 模型，该模型是在[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)中的[Iris 数据集](https://www.tensorflow.org/datasets/catalog/iris)上训练而成的。训练好的模型可以预测三种鸢尾花的类型：山鸢尾、维吉尼亚鸢尾或变色鸢尾。

费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [ Vertex AI 价格 ](https://cloud.google.com/vertex-ai/pricing) 和 [ Cloud Storage 价格 ](https://cloud.google.com/storage/pricing)，并使用 [ 价格计算器 ](https://cloud.google.com/products/calculator/) 根据您预计的使用情况生成费用估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他必需的软件包

In [ ]:
# Install required packages.
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 google-cloud-bigquery

重新启动运行时（仅针对Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。请等待直到它完成后再继续下一步。⚠️</b>
</div>

### 在Google Colab上验证您的笔记本环境

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google云项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 用户电子邮件

设置您的用户电子邮件地址以接收监控警报。

In [ ]:
import os

USER_EMAIL = "[your-email-addr]"  # @param {type:"string"}

if os.getenv("IS_TESTING"):
    USER_EMAIL = "noreply@google.com"

### 关于服务账号和权限的注意事项

**默认情况下不需要任何配置**，如果遇到与权限相关的问题，请确保上述服务账号具有所需的角色：

|服务账号邮箱|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|计算引擎默认服务账号|Dataflow Admin, Dataflow Worker, Storage Admin, BigQuery Admin, Vertex AI User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI平台服务代理|Vertex AI服务代理|

1. 进入[IAM控制台](https://console.cloud.google.com/iam-admin/iam)。
2. 选中**包含由谷歌提供的角色授予**复选框。
3. 找到上述邮箱。
4. 授予相应的角色。

### 使用来自不同项目的数据源
如果您正在使用来自不同项目的数据源：
- 对于BigQuery数据源，为两个服务账号授予“BigQuery数据查看器”角色。
- 对于CSV数据源，为两个服务账号授予“存储对象查看器”角色。

创建一个云存储桶

创建一个存储桶来存储中间工件，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶不存在：运行以下单元格创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import os
import random

import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

### 初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### 创建BigQuery客户端

在本教程中，您使用了与用于训练预训练模型的相同的公共BigQuery表中的数据。您创建了一个客户端接口，随后用于访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

#### 设置预构建容器

设置预构建的 Docker 容器镜像以进行预测。

要获取最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
DEPLOY_VERSION = "xgboost-cpu.1-1"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`DEPLOY_COMPUTE`以配置用于预测的计算资源。
 - 设置一个`机器类型`：
     - `n1-standard`：每个 vCPU 3.75GB 内存
     - `n1-highmem`：每个 vCPU 6.5GB 内存
     - `n1-highcpu`：每个 vCPU 0.9GB 内存
 - `vCPUs`：数量为 \[2, 4, 8, 16, 32, 64, 96\]

**注意**：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持GPU。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## Vertex AI 模型监控简介

Vertex AI 模型监控支持 AutoML 表格模型和自定义表格模型。您可以监控入站预测请求中的特征倾斜和漂移检测，或者监控出站预测响应中的特征归因（可解释 AI）。换句话说，您可以监控衡量特征对输出（预测）的贡献的归因分布。

以下是启用模型监控的基本步骤：

1. 将 Vertex AI AutoML 或自定义表格模型部署到 Vertex AI 端点。
2. 配置模型监控规范。
3. 上传模型监控规范到 Vertex AI 端点。
4. 上传架构或使用*输入模式*的自动生成进行解析。
5. 对于特征倾斜检测，上传训练数据。这样可以自动生成特征分布。
6. 对于特征归因，上传相应的*Vertex 可解释 AI*规范。

配置完成后，您可以启用或禁用监控，更改警报并更新模型监控配置。

启用模型监控后，采样的传入预测请求将被记录到 BigQuery 表中。然后分析记录请求中包含的输入特征值，以检测特定间隔的倾斜或漂移。您可以设置采样率以监视模型的一部分实际输入，并设置监控间隔。

模型监控服务需要知道如何解析特征值，这称为输入模式。对于 AutoML 表格模型，输入架构会自动生成。对于自定义表格模型，服务尝试自动从前 1000 个预测请求中推导输入模式。或者，您可以上传输入模式。

对于倾斜检测，监控服务需要训练数据中值的统计分布基线。对于 AutoML 表格模型，这会自动生成。对于自定义表格模型，您需要将训练数据上传到服务中，并让服务自动推导分布。

对于特征归因倾斜和漂移检测，您需要为已部署的自定义表格模型启用 Vertex 可解释 AI 功能。对于 AutoML 模型，Vertex 可解释 AI 会自动启用。

了解更多有关[Vertex AI 模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

将模型工件上传到Vertex AI模型注册表

首先，使用`upload()`方法将预训练的XGBoost表格模型工件作为Vertex AI模型资源上传，使用以下参数：

- `display_name`：模型资源的可读名称。
- `artifact_uri`：模型工件的云存储位置。
- `serving_container_image`：模型部署到Vertex AI端点资源时要使用的服务容器映像。
- `sync`：是否等待过程完成，或立即返回（异步）。

In [ ]:
MODEL_ARTIFACT_URI = (
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/xgboost_iris"
)

model = aiplatform.Model.upload(
    display_name="xgboost_iris",
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

### 部署模型到终端

接下来，使用`deploy()`方法将您的Vertex AI模型资源部署到Vertex AI终端资源，使用以下参数：

- `deploy_model_display`: 部署模型的人类可读名称。
- `machine_type`: 每个VM节点实例的机器类型。
- `min_replica_count`: 自动扩展需预留的最小节点数。
- `max_replica_count`: 自动扩展需预留的最大节点数。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1


endpoint = model.deploy(
    deployed_model_display_name="xgboost_iris",
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

## 配置监控任务

配置监控任务包括以下规范：

- `alert_config`：应该接收监控警报的电子邮件地址（们）。
- `schedule_config`：分析预测的时间窗口。
- `logging_sampling_strategy`：对预测请求进行抽样的速率。
- `drift_config`：要监视的特征和漂移阈值。
- `skew_config`：要监视的特征和偏差阈值。

### 配置警报规范

使用以下设置配置`alerting_config`规范：

- `user_emails`：应接收警报的一个或多个电子邮件列表。
- `enable_logging`：将检测到的异常流式传输到Cloud Logging。默认值为False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您将使用以下设置配置`schedule_config`规范：

- `monitor_interval`: 设置模型监控作业的调度间隔，单位为小时。最小时间间隔为1小时。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置采样规范

接下来，您可以使用以下设置配置`logging_sampling_strategy`规范：

- `sample_rate`: 作为百分比（介于0和1之间）的速率，用于随机抽样监视预测请求。选定的样本将被记录到一个BigQuery表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您可以使用以下设置来配置 `drift_config` 规范：

- `drift_thresholds`: 一个键/值对字典，其中键是要监控漂移的输入特征，值是检测阈值。如果未指定，则特征的默认漂移阈值为 0.3（30%）。

*注意:* 启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "sepal_length": DRIFT_THRESHOLD_VALUE,
    "petal_length": DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 组装客观规范

最后，您将客观规范 `objective_config` 与以下设置进行组装：

- `skew_detection_config`：（可选）用于倾斜检测配置的规范。
- `drift_detection_config`：（可选）用于漂移检测配置的规范。
- `explanation_config`：（可选）在启用特征归因监视时用于解释的规范。

*注意：*您不需要配置倾斜检测，因为假设您无法访问训练数据。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=None,
    drift_detection_config=drift_config,
    explanation_config=None,
)

### 创建输入模式

监控服务需要了解模型的输入特性和数据类型，这被称为*输入模式*。*输入模式*可以是以下两种之一：
 - 预加载到监控服务。
 - 在接收到前1000个预测实例后，由监控服务自动生成。

在本教程中，您将预加载*输入模式*。

#### 创建预定义的输入模式

预定义的*输入模式*以YAML文件指定。在这个例子中，您将根据模型的输入层生成YAML规范。在这种情况下，输入层是一个由四个浮点数值组成的数组。在模式中，这表示为：

- `type: array`：指示输入是一个数组（列表）
- `properties`：数组中输入的有序列表
- `properties -> name`：对应数组中值的别名（例如，sepal_length）。
- `properties -> type: number`：数组元素的值为浮点数。
- `required`：由别名指定的数组中值的顺序。

然后，输入模式告知模型监控服务如何将未命名的输入值映射到相应的特性别名，然后可以在您的模型监控配置中指定这些别名。

预定义的`输入模式`必须加载到云存储位置。

了解更多关于[解析输入的自定义实例模式](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas)。

In [ ]:
yaml = """type: array
properties:
  sepal_length:
    type: number
  sepal_width:
    type: number
  petal_length:
    type: number
  petal_width:
    type: number
required:
  - sepal_length
  - sepal_width
  - petal_length
  - petal_width
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

### 创建监控作业

使用Vertex AI的`ModelDeploymentMonitoringJob.create()`方法创建一个监控作业，使用您的监控规范，并使用以下参数：

- `display_name`：监控作业的可读名称。
- `project`：项目ID。
- `location`：位置。
- `endpoint`：要启用监控的Vertex AI端点的完全限定资源名称。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。
- `analysis_instance_schema_uri`：包含*输入模式*的YAML文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="xgboost_iris",
    project=PROJECT_ID,
    location=LOCATION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

模型监控作业的电子邮件通知。

电子邮件通知将发送到警报配置中的电子邮件地址，通知您模型监控作业现在已启用。

电子邮件的内容如下：

<blockquote>
您好，Vertex AI 客户，

您收到此邮件是因为您正在使用 Vertex AI 模型监控服务。
此邮件是通知您我们已收到您设置漂移或偏差检测的请求，用于下面列出的预测端点。从现在开始，传入的预测请求将被采样并记录以进行分析。
原始请求和响应将从预测服务中收集并保存在 bq://[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict 。
</blockquote>

#### 监控作业状态

在启动Vertex AI模型监视作业之后，它会保持在**PENDING**状态，直到计算出`偏差分布基线`。监视服务会启动一个批处理作业，从训练数据生成分布基线。

一旦基线分布生成完成，监视作业将变为**OFFLINE**状态。每隔一段时间，例如每小时一次，监视作业会进入**RUNNING**状态，分析采样数据。完成后，它将回到**OFFLINE**状态，并等待下一次计划的分析。

In [ ]:
jobs = monitoring_job.list(filter="display_name=xgboost_iris")
job = jobs[0]
print(job.state)

暂停一下，等待监控任务启用。

In [ ]:
import time

time.sleep(180)

### 生成第一个基准线的合成预测请求

接下来，您需要创建 1000 个合成数据项，用于预测请求。

In [ ]:
instances = []
for _ in range(1000):
    sepal_length = random.uniform(0.5, 3.5)
    sepal_width = random.uniform(0.2, 2.0)
    petal_length = random.uniform(0.5, 2.0)
    petal_width = random.uniform(0.2, 1.5)
    instances.append([sepal_length, sepal_width, petal_length, petal_width])

### 发送预测请求

接下来，使用`predict()`方法将1000个预测请求发送到您的Vertex AI端点资源。

请注意，模型输出的类别是一个浮点数值。例如，`0.0`表示标签`0`。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

在下一个监控间隔中，抽样预测结果将被复制到BigQuery的日志表中。一旦条目进入BigQuery表，监控服务就会分析抽样数据。

接下来，您需要等待第一个记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，抽样率为50%，您应该会看到大约500条记录。

*注意*：这需要占用监控间隔的时间（例如，一个小时）。

In [ ]:
import time

while True:

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break
    time.sleep(180)

### 生成用于漂移检测的合成预测请求

修改数据（合成数据）以触发从先前基线分布到当前分布的预测请求中的漂移检测，具体操作如下：

- `sepal_length`：将数值增加4倍。

In [ ]:
instances = []
for _ in range(1000):
    sepal_length = random.uniform(0.5, 3.5) * 4.0
    sepal_width = random.uniform(0.2, 2.0)
    petal_length = random.uniform(0.5, 2.0)
    petal_width = random.uniform(0.2, 1.5)
    instances.append([sepal_length, sepal_width, petal_length, petal_width])

发出预测请求

接下来，您将使用`predict()`方法向您的Vertex AI端点资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 在监控过程中的漂移检测

特征输入漂移检测会在下一个监控间隔期间发生。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业将从**离线**状态转为**运行**状态。在运行过程中，它会分析在此间隔期间从预测中记录的样本表，并将其与上一个监控间隔的分布进行比较。

一旦分析完成，监控作业会通过电子邮件通知检测到的漂移，即`cnt_user_engagement`。然后，监控作业将进入**离线**状态，直到下一个间隔期间。

#### 等待监控间隔

从监控间隔的分析完成时起，直到您收到电子邮件警报可能需要40分钟或更长时间。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 记录抽样请求

在下一个监控间隔中，抽样预测结果将被复制到BigQuery日志表中。一旦条目进入BigQuery表中，监控服务将分析抽样数据。

接下来，您需要等待第二个已记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，带有50%的抽样率，您应该会看到大约1000条记录。

*注意*：这将占据监控间隔的整个时间（例如一个小时）。

In [ ]:
import time

while True:

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 950:
        break
    time.sleep(180)

### 删除监控作业

一旦您收到了电子邮件提醒并验证了内容，您可以：
- 使用`pause（）`方法暂停监控作业。
- 使用`delete（）`方法删除监控作业。

In [ ]:
# Pause the job
monitoring_job.pause()
# Delete the job
monitoring_job.delete()

清除

要清除本项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy the model from endpoint before deletion
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the Cloud Storage bucket
delete_bucket = False
if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}

# Delete the locally generated files
! rm -f schema.yaml

# Delete the BigQuery table
! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}